#1 Carga del dataset

In [12]:
import pandas as pd

df = pd.read_csv('houses_to_buy_v2.csv', sep=',', encoding='utf-8')

#3.1 Obtencion de coordinadas gps

In [13]:
import requests
import math

def clean_address(address):
  to_remove = ["Piso en alquiler en", "Apartamento en alquiler en", "Ático en alquiler en", 
              "Dúplex en alquiler en", "Casa adosada en alquiler en", "Chalet en alquiler en",
              "Casa unifamiliar en alquiler", "Estudio en alquiler en", "Loft en alquiler en",
              "Casa pareada en alquiler en", "Chalet unifamiliar en alquiler en", "Chalet adosado en alquiler en", "Casa en alquiler en",
              "Finca rústica en alquiler en", "Chalet pareado en alquiler en "]

  for substring in to_remove:
    address = address.replace(substring, "")

  address = address.split("cerca")[0]

  return address

def get_coordinates():
  url = 'http://api.positionstack.com/v1/forward'

  for index, row in df.iterrows():
    row = df.iloc[index]

    if "latitude" in df.columns and not math.isnan(row["latitude"]): continue

    address = clean_address(row['title'])
    city = row["city"]
    region = city + ', España'
    params = {
      'access_key': 'e8ab79b0edab30e9b58f2773fa1caf5e',
      'query': address,
      'limit': 1,
      "region": region
    }

    response = requests.get(url, params=params)
    location = response.json()['data']

    if len(location) > 0:
      location = location[0] 
      if not location: continue
      print(location)
      df.at[index, 'latitude'] = location['latitude']
      df.at[index, 'longitude'] = location['longitude']
      print(index, location["latitude"], location["longitude"], address, region)
    else:
      print(f"{index} Could not find location for address: {address} {region}")

get_coordinates()


1 Could not find location for address: Piso en venta en Arroyo de La Miel malaga, España
3 Could not find location for address: Casa en venta en La Gaspara-Bahía Dorada-Buenas Noches malaga, España
4 Could not find location for address: Apartamento en venta en Fuengirola malaga, España
6 Could not find location for address: Casa en venta en Viñuela malaga, España
7 Could not find location for address: Apartamento en venta en Casco Antiguo malaga, España
8 Could not find location for address: Apartamento en venta en The Golden Mile malaga, España
9 Could not find location for address: Apartamento en venta en Casco Antiguo malaga, España
11 Could not find location for address: Apartamento en venta en Fuengirola malaga, España
13 Could not find location for address: Chalet en venta en Los Flamingos malaga, España
15 Could not find location for address: Ático en venta en Mijas Costa malaga, España
17 Could not find location for address: Chalet en venta en Torremuelle malaga, España
18 Coul

In [15]:
df.to_csv('houses_v2.csv', index=False)
print(len(df))
print(len(df[df['latitude'].isna()]))
print(len(df)-len(df[df['latitude'].isna()]))

2715
1194
1521
